In [1]:
import torch
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
class Node:
    def __init__(self, gini, num_samples, num_samples_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

In [3]:
class DecisionTree_CART():
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        """Build decision tree classifier
        :argument X: Input Tensor
        :argument y: ground truth Tensor
        :variable n_classes_: Number of Classes in target variable
        :variable n_features_: Number of features
        :variable tree_: Making decision tree based on X, y along with max_depth
        """
        self.n_classes_ = len(y.unique())  # classes are assumed to go from 0 to n-1
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)

    def _gini(self, y):
        """Compute Gini impurity of a non-empty node.
        Gini impurity is defined as Σ p(1-p) over all classes, with p the frequency of a
        class within the node. Since Σ p = 1, this is equivalent to 1 - Σ p^2.

        :var m: Sample Size
        """
        m = y.shape[0]

        return 1.0 - sum((torch.sum(y == c).item() // m) ** 2 for c in range(self.n_classes_))

    def _best_split(self, X, y):
        """Find the best split for a node.
        "Best" means that the average impurity of the two children, weighted by their
        population, is the smallest possible. Additionally it must be less than the
        impurity of the current node.
        To find the best split, we loop through all the features, and consider all the
        midpoints between adjacent training samples as possible thresholds. We compute
        the Gini impurity of the split generated by that particular feature/threshold
        pair, and return the pair with smallest impurity.
        Returns:
            best_idx: Index of the feature for best split, or None if no split is found.
            best_thr: Threshold to use for the split, or None if no split is found.
        """
        # Need at least two elements to split a node.
        m = y.shape[0]
        if m <= 1:
            return None, None

        # Count of each class in the current node.
        num_parent = [torch.sum(y == c).item() for c in range(self.n_classes_)]
        print(f'num_parent {num_parent}')

        # Gini of current node.
        best_gini = 1.0 - sum((n // m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None

        # Loop through all features.
        for idx in range(self.n_features_):
            # Sort data along selected feature.
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))

            # We could actually split the node according to each feature/threshold pair
            # and count the resulting population for each class in the children, but
            # instead we compute them in an iterative fashion, making this for loop
            # linear rather than quadratic.
            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()
            for i in range(1, m):  # possible split positions
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] // (m - i)) ** 2 for x in range(self.n_classes_)
                )

                # The Gini impurity of a split is the weighted average of the Gini
                # impurity of the children.
                gini = (i * gini_left + (m - i) * gini_right) / m

                # The following condition is to make sure we don't try to split two
                # points with identical values for that feature, as it is impossible
                # (both have to end up on the same side of a split).
                if thresholds[i] == thresholds[i - 1]:
                    continue

                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2  # midpoint

        print("Best Index and Threshold",best_idx, best_thr)

        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        """Build a decision tree by recursively finding the best split."""
        # Population for each class in current node. The predicted class is the one with
        # largest population.
        num_samples_per_class = torch.tensor([torch.sum(y == i).item() for i in range(self.n_classes_)])
        predicted_class = torch.argmax(num_samples_per_class)
        node = Node(
            gini=self._gini(y),
            num_samples=y.shape[0],
            num_samples_per_class=num_samples_per_class,
            predicted_class=predicted_class,
        )

        # Split recursively until maximum depth is reached.
        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _predict(self, inputs):
        """Predict class for a single sample
        A tree with threshold value set on each node from root to one level above leaf node is built.
        """
        node = self.tree_
        while node.right:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

if __name__ == "__main__":
    """
    :variable X: Input tensor with 30 features
    :target y: Output tensor with 2 classes
    
    * Converting Numpy array into torch tensor.
    * Creating DecisionTree Object with max_depth 5.
    * Fit and predict with DecisionTree Object.
    """
    breast_cancer = load_breast_cancer()
    X = breast_cancer['data']
    y = breast_cancer['target']
    X = torch.tensor(X)
    y = torch.tensor(y)

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    classifier = DecisionTree_CART(max_depth=5)
    classifier.fit(x_train, y_train)
    y_predict = classifier.predict(x_test)

    print(f'Accuracy: {accuracy_score(y_test, y_predict)}')

num_parent [164, 291]
Best Index and Threshold 22 tensor(127.2000, dtype=torch.float64)
num_parent [58, 291]
Best Index and Threshold 27 tensor(0.1712, dtype=torch.float64)
num_parent [32, 291]
Best Index and Threshold 23 tensor(957.4500, dtype=torch.float64)
num_parent [20, 289]
Best Index and Threshold 22 tensor(116.0500, dtype=torch.float64)
num_parent [18, 289]
Best Index and Threshold 7 tensor(0.0878, dtype=torch.float64)
num_parent [2, 0]
Best Index and Threshold None None
num_parent [12, 2]
Best Index and Threshold 0 tensor(16.3800, dtype=torch.float64)
num_parent [12, 0]
Best Index and Threshold None None
num_parent [0, 2]
Best Index and Threshold None None
num_parent [26, 0]
Best Index and Threshold None None
num_parent [106, 0]
Best Index and Threshold None None
Accuracy: 0.9210526315789473
